Import the necessary libraries and modules.

In [1]:
import pandas as pd
import csv

Change the Path variable to point to your CSV file

In [2]:
path = r"C:\Users\joacosta\Dev\Python\ORF5\Data\VS_A_9-19-2025_ItemInducted_Test-2.csv"

In [3]:
raw_df = pd.read_csv(
    path,
    sep=";",
    header=None,
    engine="python",
    quoting=csv.QUOTE_NONE,
    skipinitialspace=True,
    on_bad_lines="skip",
    dtype=str
)

# Cleaning dataset

In [4]:
df_no_quotation = raw_df.replace('"', '', regex=True)
df_no_quotation.head()

,0,1,2,3,4,5,6,7,8,9,10,11
0,250919 085754 627,N,SMC,10.158.244.100:7200,AWCS.Comm,,,"20:39:17,943",MC01,54123,SQ 181,"->{event: AwcsConverterReceive40ItemInducted, ..."
1,250919 085755 994,N,SMC,10.158.244.100:7200,AWCS.Comm,,,"20:39:19,344",MC01,54123,SQ 216,"->{event: AwcsConverterReceive40ItemInducted, ..."
2,250919 085757 298,N,SMC,10.158.244.100:7200,AWCS.Comm,,,"20:39:20,593",MC01,54123,SQ 250,"->{event: AwcsConverterReceive40ItemInducted, ..."
3,250919 085757 832,N,SMC,10.158.244.100:7200,AWCS.Comm,,,"20:39:21,184",MC01,54123,SQ 036,"->{event: AwcsConverterReceive40ItemInducted, ..."
4,250919 085758 242,N,SMC,10.158.244.100:7200,AWCS.Comm,,,"20:39:21,594",MC01,54123,SQ 042,"->{event: AwcsConverterReceive40ItemInducted, ..."


In [5]:
# Rename original df columns
df_no_quotation.columns = [
    "timeStamp", "flag", "systemName", "ipAddress", "sender", "unkown",
    "unkown_2", "PLCTimeStamp", "machineCode", "unitID", "sequenceNo",
    "rawMessage"
]

In [6]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', 0)
pd.set_option('display.max_rows', None)

print(df_no_quotation.iloc[0,11])

->{event: AwcsConverterReceive40ItemInducted, awcsAction: [ScanItem], plcRecordNo: 0318, itemID: 318U, indexNo: 0318, awcsStateNow: USED, awcsStateNew: INDUCTED, inductionStatus: 0, inductionNo: 6, destinationNo: 4099, carrierNo: 24, carrierCount: 1, comHost: AwcsConverterMcIf, comMode: Receive, telegramType: 40}<


In [7]:
df_no_quotation.head(1)

,timeStamp,flag,systemName,ipAddress,sender,unkown,unkown_2,PLCTimeStamp,machineCode,unitID,sequenceNo,rawMessage
0,250919 085754 627,N,SMC,10.158.244.100:7200,AWCS.Comm,,,"20:39:17,943",MC01,54123,SQ 181,"->{event: AwcsConverterReceive40ItemInducted, awcsAction: [ScanItem], plcRecordNo: 0318, itemID: 318U, indexNo: 0318, awcsStateNow: USED, awcsStateNew: INDUCTED, inductionStatus: 0, inductionNo: 6, destinationNo: 4099, carrierNo: 24, carrierCount: 1, comHost: AwcsConverterMcIf, comMode: Receive, telegramType: 40}<"


In [8]:
df_no_quotation["rawMessage"] = df_no_quotation["rawMessage"].str.removeprefix("->{").str.removesuffix("}<")
print(df_no_quotation.iloc[0,11])

event: AwcsConverterReceive40ItemInducted, awcsAction: [ScanItem], plcRecordNo: 0318, itemID: 318U, indexNo: 0318, awcsStateNow: USED, awcsStateNew: INDUCTED, inductionStatus: 0, inductionNo: 6, destinationNo: 4099, carrierNo: 24, carrierCount: 1, comHost: AwcsConverterMcIf, comMode: Receive, telegramType: 40


In [9]:
def split_key_values(text):
    parts = []
    buf = ""
    inside_brackets = 0
    
    for ch in text:
        if ch == "[":
            inside_brackets += 1
        elif ch == "]":
            inside_brackets -= 1
        
        if ch == "," and inside_brackets == 0:
            parts.append(buf.strip())
            buf = ""
        else:
            buf += ch
    if buf:
        parts.append(buf.strip())
    return parts

def parse_row(text):
    items = split_key_values(text)
    d = {}
    for item in items:
        if ":" in item:
            k, v = item.split(":", 1)
            d[k.strip()] = v.strip()
    return d

df_expanded = df_no_quotation["rawMessage"].apply(parse_row).apply(pd.Series)

df_expanded.head()


,event,awcsAction,plcRecordNo,itemID,indexNo,awcsStateNow,awcsStateNew,inductionStatus,inductionNo,destinationNo,carrierNo,carrierCount,comHost,comMode,telegramType
0,AwcsConverterReceive40ItemInducted,[ScanItem],0318,318U,0318,USED,INDUCTED,0,6,4099,24,1,AwcsConverterMcIf,Receive,40
1,AwcsConverterReceive40ItemInducted,[ScanItem],0317,317U,0317,USED,INDUCTED,0,6,4099,30,1,AwcsConverterMcIf,Receive,40
2,AwcsConverterReceive40ItemInducted,[ScanItem],0319,319U,0319,USED,INDUCTED,0,6,4099,35,1,AwcsConverterMcIf,Receive,40
3,AwcsConverterReceive40ItemInducted,[ScanItem],0320,320U,0320,USED,INDUCTED,0,4,4099,62,1,AwcsConverterMcIf,Receive,40
4,AwcsConverterReceive40ItemInducted,[ScanItem],0321,321U,0321,USED,INDUCTED,0,6,4099,39,1,AwcsConverterMcIf,Receive,40


In [10]:
# Drop the old structured column
df_temp = df_no_quotation.drop(columns=["rawMessage"])

# Merge with expanded dataframe
df_clean = pd.concat([df_temp, df_expanded], axis=1)

df_clean.head(10)

,timeStamp,flag,systemName,ipAddress,sender,unkown,unkown_2,PLCTimeStamp,machineCode,unitID,...,awcsStateNow,awcsStateNew,inductionStatus,inductionNo,destinationNo,carrierNo,carrierCount,comHost,comMode,telegramType
0,250919 085754 627,N,SMC,10.158.244.100:7200,AWCS.Comm,,,"20:39:17,943",MC01,54123,...,USED,INDUCTED,0,6,4099,24,1,AwcsConverterMcIf,Receive,40
1,250919 085755 994,N,SMC,10.158.244.100:7200,AWCS.Comm,,,"20:39:19,344",MC01,54123,...,USED,INDUCTED,0,6,4099,30,1,AwcsConverterMcIf,Receive,40
2,250919 085757 298,N,SMC,10.158.244.100:7200,AWCS.Comm,,,"20:39:20,593",MC01,54123,...,USED,INDUCTED,0,6,4099,35,1,AwcsConverterMcIf,Receive,40
3,250919 085757 832,N,SMC,10.158.244.100:7200,AWCS.Comm,,,"20:39:21,184",MC01,54123,...,USED,INDUCTED,0,4,4099,62,1,AwcsConverterMcIf,Receive,40
4,250919 085758 242,N,SMC,10.158.244.100:7200,AWCS.Comm,,,"20:39:21,594",MC01,54123,...,USED,INDUCTED,0,6,4099,39,1,AwcsConverterMcIf,Receive,40
5,250919 085759 134,N,SMC,10.158.244.100:7200,AWCS.Comm,,,"20:39:22,484",MC01,54123,...,USED,INDUCTED,0,6,4099,43,1,AwcsConverterMcIf,Receive,40
6,250919 085759 251,N,SMC,10.158.244.100:7200,AWCS.Comm,,,"20:39:22,603",MC01,54123,...,USED,INDUCTED,0,4,4099,68,1,AwcsConverterMcIf,Receive,40
7,250919 085800 364,N,SMC,10.158.244.100:7200,AWCS.Comm,,,"20:39:23,686",MC01,54123,...,USED,INDUCTED,0,6,4099,48,1,AwcsConverterMcIf,Receive,40
8,250919 085800 524,N,SMC,10.158.244.100:7200,AWCS.Comm,,,"20:39:23,838",MC01,54123,...,USED,INDUCTED,0,4,4099,73,1,AwcsConverterMcIf,Receive,40
9,250919 085800 617,N,SMC,10.158.244.100:7200,AWCS.Comm,,,"20:39:23,944",MC01,54123,...,USED,INDUCTED,0,5,4099,61,1,AwcsConverterMcIf,Receive,40


Removing bad packages that just produce noise in the results

In [11]:
print(df_clean["indexNo"].dtype)

object


In [12]:
# 1) Crear los 5 datasets filtrando por inductionNo
targets = [4, 5, 6, 100, 101]
mini = {v: df_clean[df_clean["inductionNo"] == v].copy() for v in targets}

# Acceso: mini[4], mini[5], mini[6], mini[100], mini[101]

# 2) Extra: cuántos y cuáles valores distintos hay en inductionNo
unique_vals = sorted(df_clean["inductionNo"].unique().tolist())
num_unique = len(unique_vals)
print("Valores distintos en 'inductionNo':", unique_vals)
print("Total de valores distintos:", num_unique)


Valores distintos en 'inductionNo': ['4', '5', '6']
Total de valores distintos: 3


In [13]:
import pandas as pd
import datetime as dt
import math

def analyze_dataset(df: pd.DataFrame,
                    start_time_24h="16:00",     # inicio de ventana (24h)
                    window_minutes=10,          # duración de ventana (min)
                    TARGET_PPH=2450,
                    line_name="IU006",
                    measuring_point="MP1000605"):
    # Asegurar timestamp
    if "timestamp" not in df.columns:
        if "stamp_raw" not in df.columns:
            raise KeyError("Falta 'timestamp' y 'stamp_raw' en el DataFrame.")
        sr = df["stamp_raw"].astype(str).str.replace(" ", "", regex=False)  # p.ej. 250917160001278
        df = df.copy()
        df["timestamp"] = pd.to_datetime(sr, format="%y%m%d%H%M%S%f", errors="coerce")

    valid = df["timestamp"].notna()
    if not valid.any():
        return {"error": "Sin timestamps válidos"}

    start_time = df.loc[valid, "timestamp"].min()
    end_time   = df.loc[valid, "timestamp"].max()
    general_delta_time = end_time - start_time
    delta_secs = general_delta_time.total_seconds()
    delta_min = delta_secs / 60.0 if delta_secs > 0 else float("nan")

    # Ventana
    base_date = df["timestamp"].min().normalize()
    try:
        t = dt.time.fromisoformat(start_time_24h)
    except ValueError:
        t = dt.time(int(start_time_24h), 0, 0)

    win_start = pd.Timestamp(dt.datetime.combine(base_date.date(), t))
    win_end   = win_start + pd.Timedelta(minutes=window_minutes)
    mask      = (df["timestamp"] >= win_start) & (df["timestamp"] <= win_end)
    window_df = df.loc[mask]

    n_items = len(window_df)
    sec_per_bag   = (delta_secs / n_items) if n_items > 0 and delta_secs > 0 else float("nan")
    items_per_sec = (n_items / delta_secs) if delta_secs > 0 else float("nan")
    items_per_min = items_per_sec * 60 if delta_secs > 0 else float("nan")

    # PPH
    pph_overall = n_items * 3600 / delta_secs if delta_secs > 0 else float("nan")
    window_secs = (win_end - win_start).total_seconds()
    pph_window  = n_items * 3600 / window_secs if window_secs > 0 else float("nan")

    passed = (pph_window >= TARGET_PPH) if window_secs > 0 and n_items > 0 else False
    needed_items_for_pass = math.ceil(TARGET_PPH * window_secs / 3600) if window_secs > 0 else float("nan")
    items_short = max(0, needed_items_for_pass - n_items) if window_secs > 0 else float("nan")

    spb_current = window_secs / n_items if n_items > 0 else float("inf")
    spb_target  = 3600 / TARGET_PPH

    recommendation = (
        f"Need +{items_short} items in this {window_minutes:.1f} min window "
        f"or reduce sec/bag to ≤ {spb_target:.2f} (current {spb_current:.2f})."
    ) if (window_secs > 0 and n_items > 0 and items_short > 0) else "On target or above. Maintain current rate."

    meridiem = win_end.strftime('%p')
    time_label = f"{win_start:%H%M}-{win_end:%H%M}{meridiem}"
    summary_line = f"{line_name};Measuring Point: {measuring_point}: Time {time_label}; Total Inducted = {n_items}"

    return {
        "win_start": win_start, "win_end": win_end,
        "n_items": n_items,
        "delta_secs": delta_secs, "delta_min": delta_min,
        "sec_per_bag": sec_per_bag,
        "items_per_sec": items_per_sec, "items_per_min": items_per_min,
        "pph_overall": pph_overall, "pph_window": pph_window,
        "TARGET_PPH": TARGET_PPH, "passed": passed,
        "needed_items_for_pass": needed_items_for_pass, "items_short": items_short,
        "spb_current": spb_current, "spb_target": spb_target,
        "recommendation": recommendation,
        "summary_line": summary_line,
        "valid_rows": int(valid.sum()), "invalid_rows": int(len(df)-valid.sum()),
        "start_time": start_time, "end_time": end_time, "general_delta_time": general_delta_time,
        "window_df": window_df,  # si quieres inspeccionarlo
    }

# ---- APLICAR A TUS 5 MINI-DATASETS EN MEMORIA ----
# Asumiendo que ya creaste: mini = {4: df4, 5: df5, 6: df6, 100: df100, 101: df101}

results = {
    k: analyze_dataset(g,
                       start_time_24h="16:00",   # ajusta si quieres
                       window_minutes=10,        # ajusta si quieres
                       TARGET_PPH=2450,
                       line_name=f"IND{k}",
                       measuring_point="MP1000605")
    for k, g in mini.items()
}

# Ejemplos de acceso:
# results[4]["pph_window"], results[5]["summary_line"], results[6]["window_df"].head()


KeyError: "Falta 'timestamp' y 'stamp_raw' en el DataFrame."